In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import math
from datetime import datetime
from src.heikin_ashi import heikin_ashi
from src.indicators import supertrend, VWAP2, smc, wavetrend3d, atr
import pandas_ta as ta

In [3]:
auxusd = pd.read_csv("./data_directory/AUXUSD.csv")
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'], unit='ms')
auxusd['timestamp'] = pd.to_datetime(auxusd['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
auxusd.set_index('timestamp', inplace=True)
auxusd.drop_duplicates(inplace=True)

In [4]:
eurgbp = pd.read_csv("./data_directory/EURGBP.csv")
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'], unit='ms')
eurgbp['timestamp'] = pd.to_datetime(eurgbp['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
eurgbp.set_index('timestamp', inplace=True)
eurgbp.drop_duplicates(inplace=True)

In [3]:
forex_data = pd.read_csv("./data_directory/EURUSD_2021_2023.csv")
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'], unit='ms')
forex_data['timestamp'] = pd.to_datetime(forex_data['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S')))
forex_data.set_index('timestamp', inplace=True)
forex_data.drop_duplicates(inplace=True)

In [6]:
eurusd = pd.read_csv("./data_directory/EURUSD_NEW.csv")
eurusd.set_index("Date", inplace=True)
eurusd.index = pd.to_datetime(eurusd.index)
eurusd.drop_duplicates(inplace=True)

In [7]:
# Der choCH wird hier programmiert

def choCH(data, length, band):
    chocolad = pd.Series(index=data.index)
    ob = smc(data, length, band, "30T")
    prevvalue = 0
    orderblocks = []
    i = 0
    if(band == 0):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price > orderblocks[i-2][1] and orderblocks[i-1][1] < orderblocks[i-2][1] and orderblocks[i-2][1] < orderblocks[i-3][1] and orderblocks[i-3][1] < orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    elif(band == 1):
        for index, values in ob.iteritems():
            if(values > 0 and values != prevvalue):
                prevvalue = values
                give = [i, values]
                orderblocks.append(give)
            i += 1
        for i, (candle, price) in enumerate(orderblocks):
            if price < orderblocks[i-2][1] and orderblocks[i-1][1] > orderblocks[i-2][1] and orderblocks[i-2][1] > orderblocks[i-3][1] and orderblocks[i-3][1] > orderblocks[i-4][1]:
                chocolad.iloc[candle] = price
    return chocolad
    





/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/3134265937.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  chocolad = pd.Series(index=data.index)
/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/3134265937.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, values in ob.iteritems():


In [8]:

for index, value in hello.iteritems():
    if value > 0:
        print(index)

2021-06-20 20:00:00
2021-07-05 04:00:00
2021-07-22 01:00:00
2021-10-14 05:30:00
2022-01-28 15:30:00
2022-02-15 17:30:00
2022-03-07 22:00:00
2022-04-11 06:30:00
2022-04-19 15:00:00
2022-04-28 16:00:00
2022-05-09 06:30:00
2022-05-16 03:00:00
2022-06-30 19:00:00
2022-09-22 15:00:00
2022-10-24 09:30:00
2022-11-29 08:00:00
2022-12-07 16:30:00
2023-01-09 22:00:00
2023-01-19 02:00:00
2023-02-20 00:30:00
2023-02-24 01:30:00


/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_28160/2810368415.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in hello.iteritems():


In [116]:


def supertrend(data, lookback, multiplier, band):
    high = data["High"]
    low = data["Low"]
    close = data["Close"]
    # ATR

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()

    # H/L AVG AND BASIC UPPER & LOWER BAND

    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).fillna(method="ffill").dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()

    # FINAL UPPER BAND
    final_bands = pd.DataFrame(columns = ['upper', 'lower'], index=upper_band.index)
    final_bands["upper"] = 0.0
    final_bands["lower"] = 0.0

    final_bands_num = final_bands["upper"].values
    final_bands_numl = final_bands["lower"].values

    # FINAL UPPER BAND

    for i in range(len(final_bands)):
        if i == 0:
            final_bands_num[i] = 0
        else:
            if (upper_band[i] < final_bands_num[i-1]) | (close[i-1] > final_bands_num[i-1]):
                final_bands_num[i] = upper_band[i]
            else:
                final_bands_num[i] = final_bands_num[i-1]

    # FINAL LOWER BAND

    for i in range(len(final_bands)):
        if i == 0:
            final_bands_numl[i] = 0
        else:
            if (lower_band[i] > final_bands_numl[i-1]) | (close[i-1] < final_bands_numl[i-1]):
                final_bands_numl[i] = lower_band[i]
            else:
                final_bands_numl[i] = final_bands_numl[i-1]

    # Supertrend

    supertrend = pd.DataFrame(columns = ['supertrend'], index=upper_band.index)
    supertrend["supertrend"] = 0.0

    supertrend_num = supertrend["supertrend"].values

    for i in range(1, len(supertrend)):
        if i == 0:
            supertrend_num[i] = 0
        elif supertrend_num[i-1] == final_bands_num[i-1] and close[i] < final_bands_num[i]:
            supertrend_num[i] = final_bands_num[i]
        elif supertrend_num[i-1] == final_bands_num[i-1] and close[i] > final_bands_num[i]:
            supertrend_num[i] = final_bands_numl[i]
        elif supertrend_num[i-1] == final_bands_numl[i-1] and close[i] > final_bands_numl[i]:
            supertrend_num[i] = final_bands_numl[i]
        elif supertrend_num[i-1] == final_bands_numl[i-1] and close[i] < final_bands_numl[i]:
            supertrend_num[i] = final_bands_num[i]

    # ST UPTREND/DOWNTREND

    upt = []
    dt = []

    for i in range(len(supertrend)):
        if close[i] > supertrend_num[i]:
            upt.append(supertrend_num[i])
            dt.append(np.nan)
        elif close[i] < supertrend_num[i]:
            upt.append(np.nan)
            dt.append(supertrend_num[i])
        else:
            upt.append(np.nan)
            dt.append(np.nan)

    new_index = pd.date_range(start= data.index[0] , periods=1, freq='3min')
    st = pd.Series(supertrend_num, index=data.index)
    upt = pd.Series(upt, index=data.index)
    dt = pd.Series(dt, index=data.index)
    upt = pd.concat([pd.Series([np.nan], index=new_index), upt])
    dt = pd.concat([pd.Series([np.nan], index=new_index), dt])

    dt = dt[~dt.index.duplicated()]
    upt = upt[~upt.index.duplicated()]

    if(band == 0):
        return dt
    elif(band == 1):
        return upt

supertrend(forex_data, 10, 2.5, 0)

2021-06-20 19:00:00         NaN
2021-06-20 19:09:00    1.186345
2021-06-20 19:36:00    1.186086
2021-06-20 19:42:00    1.186086
2021-06-20 19:45:00    1.186033
                         ...   
2023-05-19 20:51:00         NaN
2023-05-19 20:54:00         NaN
2023-05-19 20:57:00    1.081127
2023-05-19 21:00:00         NaN
2023-05-19 21:03:00         NaN
Length: 238830, dtype: float64

In [61]:
def supertrend(data, lookback=10, multiplier=2.5, band=0):
    high = data["High"]
    low = data["Low"]
    close = data["Close"]
    # ATR
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.ewm(lookback).mean()
    
    # H/L AVG AND BASIC UPPER & LOWER BAND
    
    hl_avg = (high + low) / 2
    upper_band = (hl_avg + multiplier * atr).fillna(method="ffill").dropna()
    lower_band = (hl_avg - multiplier * atr).dropna()
    
    # FINAL UPPER BAND
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in upper_band - upper_band]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i,0] = 0
        else:
            if (upper_band[i] < final_bands.iloc[i-1,0]) | (close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = upper_band[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]
    
    # FINAL LOWER BAND
    
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i, 1] = 0
        else:
            if (lower_band[i] > final_bands.iloc[i-1,1]) | (close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = lower_band[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]
    
    # SUPERTREND
    
    supertrend = pd.DataFrame(index=upper_band.index, columns=[f'supertrend_{lookback}'])
    supertrend.iloc[:, 0] = upper_band.values
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]
    
    for i in range(1, len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
    
    supertrend = supertrend.set_index(upper_band.index)
    supertrend = supertrend.dropna()[1:]
    
    # ST UPTREND/DOWNTREND
    
    upt = []
    dt = []
    close = close.iloc[len(close) - len(supertrend):]

    for i in range(len(supertrend)):
        if close[i] > supertrend.iloc[i, 0]:
            upt.append(supertrend.iloc[i, 0])
            dt.append(np.nan)
        elif close[i] < supertrend.iloc[i, 0]:
            upt.append(np.nan)
            dt.append(supertrend.iloc[i, 0])
        else:
            upt.append(np.nan)
            dt.append(np.nan)
    new_index = pd.date_range(start= data.index[0] , periods=1, freq='3min')
    st, upt, dt = pd.Series(supertrend.iloc[:, 0]), pd.Series(upt), pd.Series(dt)
    upt.index, dt.index = supertrend.index, supertrend.index
    upt = pd.concat([pd.Series([np.nan], index=new_index), upt])
    dt = pd.concat([pd.Series([np.nan], index=new_index), dt])
    data["upt"] = upt
    data["upt"].index = data.index
    data["dt"] = dt
    data["dt"].index = data.index
    if(band==0):
        return data["dt"]
    elif(band==1):
        return data["upt"]
    
hello = supertrend(forex_data, lookback=10, multiplier=2.5, band=0)


/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_30584/1024701850.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  final_bands.iloc[:,1] = final_bands.iloc[:,0]
/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_30584/1024701850.py:47: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  supertrend.iloc[:, 0] = upper_band.values


In [73]:
hello2 = supertrend(forex_data, lookback=10, multiplier=2.5, band=1)

/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_30584/1024701850.py:23: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  final_bands.iloc[:,1] = final_bands.iloc[:,0]
/var/folders/6k/qhxq2vds24v690rsqtznkwqm0000gn/T/ipykernel_30584/1024701850.py:47: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  supertrend.iloc[:, 0] = upper_band.values


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

# Parameter für Philip

atr_window = 6
einsatz = 0.5 # Hier stehen Kommazahlen für den Prozentuallen Anteil deiner Equity die du setzen willst
# Oder du kannst auch ganze Zahlen nuzen für die Anzahl der Shares die du kaufen willst. Falls du alles reinsetzen
# Willst benutz 0.9999
satr = 3 # * atr in stoploss
tatr = 9 # * atr in take profit



class PIIL(Strategy):
    def init(self):
        self.trade_opened = False
        self.take_profit = 0
        self.sl = 0
        
        self.atr = self.I(atr, self.data.df, atr_window)

    def next(self):
        if not self.trade_opened and self.data.index[-1].hour == 5 and self.data.index[-1].minute == 0:
            self.buy(size=einsatz)
            self.sl = self.data.Close[-1] - satr * self.atr[-1]
            self.take_profit = self.data.Close[-1] + tatr * self.atr[-1]
            self.trade_opened = True

        elif self.trade_opened:
            if self.data.Close[-1] > self.take_profit or self.data.Close[-1] < self.sl:
                self.position.close()
                self.trade_opened = False
                self.take_profit = 0
                self.sl = 0


class PIIS(Strategy):
    def init(self):
        self.candlecount = 0
        self.trade_opened = False
        self.take_profit = 0
        self.sl = 0
        self.atr = self.I(atr, self.data.df)

    def next(self):
        if not self.trade_opened and self.data.index[-1].hour == 5 and self.data.index[-1].minute == 0:
            self.sl = self.data.Close[-1] + satr*self.atr[-1]
            self.take_profit = self.data.Close[-1] - tatr*self.atr[-1]
            self.sell(size=einsatz)
            self.trade_opened = True

        elif self.trade_opened:
            if self.data.Close[-1] < self.take_profit or self.data.Close[-1] > self.sl:
                self.position.close()
                self.trade_opened = False
                self.take_profit = 0
                self.sl = 0

bt = Backtest(eurusd, PIIL, cash=1000000, commission=0.00, exclusive_orders=True)

output = bt.run()
print(output)
bt.plot()
